# Financial Markets Prices
---
### Autor: Diego Alegría   2023-02-24 v3
#### Basado en código de J Gonzalez
---
Código para obtener valores de ~ 257 precios relevantes para diversos reportes.
Utilizado todos los viernes después de que cierre la bolsa. (4 pm en Nueva York, 2 o 3 en Guate)

Usos primarios son un reporte semanal de Promerica el viernes, y para reporte de Commodities el lunes.

- Referencias para arreglo:
https://github.com/JECSand/yahoofinancials/blob/master/yahoofinancials/__init__.py
https://github.com/JECSand/yahoofinancials/issues/105
https://github.com/JECSand/yahoofinancials/commit/62119be23fe81f89fd5fb1bd450fc2088667c8df

---
### Instrucciones de actualizacion.

Solamente actualizar yahoofinancials con pip, El autor JECSand implementó los arreglos propuestos por GitHub.

- 257 precios son actualizados por medio de yahoofinancials.
- 10 variables son actualizadas manualmente, utilizando el excel Markets_Temporal.xlsx ubicado en DB\Weekly\Viernes Temporales\

Hay una seccion en este codigo al final que realiza el procesamiento para subir estas ultimas variables sin tener que abrir SQL.

# Importación de liberías

In [1]:
# !pip install yahoofinancials
# !pip install pyodbc

In [2]:
# import investpy
import datetime as dt
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd
import pandas_datareader.data as web
from yahoofinancials import YahooFinancials
from datetime import*
import urllib
import numpy as np
import time
import pyodbc
import traceback
import requests as requests
import numpy as np
import sys
from sqlalchemy import*

from pandas import Timestamp
style.use('ggplot')

#Fecha de actual: (Lugar para Hardcoded abajo)
fecha_inicial=datetime.today().strftime('%Y-%m-%d')
fecha_final=(datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

# Funciones par SQL

Información importante sobre esta celda

- El código de lectura trae de regreso 3 querys que se utilizan en este notebook
  - El primer query trae de regreso la última fecha
  - El segundo trae una serie de fechas que son las necesarias obtener en la base de datos para homologar si se agrega un índice nuevo
  - El tercero es un sample para crear el frame que se subirá a la base de datos

---
# Lectura de Tickets ()

Si se agrega un nuevo índice se debe agregar a esta base de datos

In [3]:
#Lectura de indices utilizados para cada variable charl
# En Source se encuentra de donde se puede obtener
try:
    df=pd.read_excel(r'CARGA NUEVOS ETFs SQL 2020.xlsx',
#     df=pd.read_excel(r'C:\Users\lijv1.LINDA_HP\CABI\Data Bases - Documents\Weekly\CARGA NUEVOS ETFs SQL 2020.xlsx',
                  sheet_name='TickersDownload')
except:
    print('Falla encontrando archivo. Intentando por D. Alegria: ')
    df=pd.read_excel(r'C:\Users\aleja\CABI\Data Bases - Documentos\Weekly\CARGA NUEVOS ETFs SQL 2020.xlsx',
                  sheet_name='TickersDownload',usecols=np.arange(0,7))
display(df)

,IdTD,IdTicker,Source,Descripcion,IdIndiceEspecifico
0,$BCOM(D),AW=F,Yahoo,Blmbg Commodity Index,3006
1,$BCOMEN(D),$BCOMEN(D),Pendiente,Blmbg Energy Sub,3775
2,$BCOMGR(D),$BCOMGR(D),Investing,Blmbg Grains Subindex,3007
3,$BCOMIN(D),$BCOMIN(D),Investing,Blmbg Industrial Mtls Sub,3008
4,$BCOMPR(D),$BCOMPR(D),Investing,Blmbg Precious Metals Sub,3009
...,...,...,...,...,...
261,XRT(HB),XRT,Yahoo,SPDR S&P Retail,2964
262,ZO=F,ZO=F,Yahoo,Oats Continuous Contract [Sep21],3022
263,ZR=F,ZR=F,Yahoo,Rough Rice,3998
264,DC=F,DC=F,Yahoo,Class III Milk,4020


## Filtro Manual de variables

In [4]:
def dia_y_mes_hoy():
    hoy = datetime.now()
    dia = hoy.day
    mes = hoy.month
    ano = hoy.year
    return dia, mes, ano

d, m, y = dia_y_mes_hoy()
print(d, m, y)

9 1 2026


In [5]:
# #Filtro de variables manual:

lista_manual = ['ERUS', 'RSX']

# #Fecha Manual (Cambiar a automático para tomar la fecha de hoy):
fecha_inicial=dt.datetime(y,m,d)
fecha_final=dt.datetime(y,12,31)

print(fecha_inicial)

2026-01-09 00:00:00


# Scrapper para Yahoo para Data TD

> Se recoge del DF anterior llamado donde están los indicadores todos aquellos donde su fuente es yahoo.com, se utiliza la librería de pandas Webreader para tomar la información.

> Debajo se crea un df_y que será el que va a contener la última lectura de cada mercado que se toma de yahoo finance y se le va agregando información que posteriormente será homologada a el formato que está en tabladatos

> Tiene un modo de fallo en donde se utiliza una vez falla la primera recolección de datos, utiliza una segunda oportunidad puesto que en el 99% de los casos itera dos veces y jala bien el indicador, si se quiere agregar más protección solo encerrar en un ciclo while, evitar usarlos por eficiencia.

In [6]:
## CELL DE INICIALIZACION Y DEFINICIONES
from tqdm.notebook import tqdm, trange     #Comentar si no se quiere utilizar progress bar
# from tqdm import tqdm, trange            #Progress bar alternativo

#Definicion de variables obtenidas desde yahoo
y=df[df['Source']=='Yahoo']
y.reset_index(inplace=True,drop=True)

#Construccion de dataframe contenedor
df_y=pd.DataFrame(columns=['IdIndiceEspecifico','Date', 'High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close'
                           ,'Descripcion', 'IdTD','IdTicker','Source'])



import datetime as dt
import yfinance as yf



#Definicion de funciones para extraccion de datos del paquete bajado:
def unpack(initial, ind):
    precios = initial

    #print("PRECIOS", precios.columns)

    #display(precios)

#     display(precios.Date)
    #Extraccion de valores de precios:
    last = pd.DataFrame({'DatoFecha':[precios['Date'][0]]})
    last['D_Open'] = precios['Open']
    last['D_High'] = precios['High']
    last['D_Low'] = precios['Low']
    last['D_Close'] = precios['Close']
    last['D_Volume'] = precios['Volume']
    last['D_OpenInterest'] = precios['Adj Close']
#     if ('Adj Close', 'AW=F') in precios.columns:
#         last['D_OpenInterest'] = precios[('Adj Close', 'AW=F')]
#     else:
#         last['D_OpenInterest'] = 0  # O cualquier otro valor predeterminado
    #Extraccion de valores de formato:
    last['IdIndiceEspecifico']=ind.IdIndiceEspecifico
    last['Descripcion']=ind.Descripcion
    last['IdTD']=ind.IdTD
    last['IdTicker']=ind.IdTicker
    last['Source']=ind.Source
    return last


def downloading_yahoo(indice,fecha_i,fecha_f):
    ind = indice.IdTicker
    yahoo_financials = yf.download(ind, fecha_i, fecha_f, auto_adjust=False)
    if yahoo_financials.empty:
        raise ValueError("El DataFrame 'yahoo_financials' está vacío. Verifica la descarga de datos.")

    yahoo_financials.reset_index(inplace=True,drop=False)

    display(yahoo_financials)

    return unpack(yahoo_financials,indice)

# if 'lista_manual' in locals:
#     y_descarga=y[y.IdTicker.isin(lista)].copy()
#     y_descarga.reset_index(inplace=True)
# else:
y_descarga = y.copy()

# Quick test
idx=y.loc[0]
display(idx.IdTicker)
last = downloading_yahoo(idx, fecha_inicial, fecha_final)
last

'AW=F'

[*********************100%***********************]  1 of 1 completed


Price,Date,Adj Close,Close,High,Low,Open,Volume
Ticker,,AW=F,AW=F,AW=F,AW=F,AW=F,AW=F
0,2026-01-09,112.110001,112.110001,112.809998,112.110001,112.419998,207


,DatoFecha,D_Open,D_High,D_Low,D_Close,D_Volume,D_OpenInterest,IdIndiceEspecifico,Descripcion,IdTD,IdTicker,Source
0,2026-01-09,112.419998,112.809998,112.110001,112.110001,207,112.110001,3006,Blmbg Commodity Index,$BCOM(D),AW=F,Yahoo


In [7]:
y_descarga

,IdTD,IdTicker,Source,Descripcion,IdIndiceEspecifico
0,$BCOM(D),AW=F,Yahoo,Blmbg Commodity Index,3006
1,$COMPX,^IXIC,Yahoo,Nasdaq Composite Index,2990
2,$INX,^GSPC,Yahoo,S&P 500 Index,2986
3,@C(D),ZC=F,Yahoo,Corn Continuous Contract [Sep21],3011
4,@CC(D),CC=F,Yahoo,Cocoa Continuous Contract [Sep21],3012
...,...,...,...,...,...
254,XRT(HB),XRT,Yahoo,SPDR S&P Retail,2964
255,ZO=F,ZO=F,Yahoo,Oats Continuous Contract [Sep21],3022
256,ZR=F,ZR=F,Yahoo,Rough Rice,3998
257,DC=F,DC=F,Yahoo,Class III Milk,4020


In [60]:
## DESCARGA COMPLETA de YAHOO
missing = [] #Indices de faltantes
a_juntar = [] #Variable para juntar dataframes
espera = 1 # seg
retries = 3

for i in trange(len(y_descarga)):     #Reemplazar con range si no se quiere utilizar progress bar
# for i in range(0,1):
    #Procesamiento original
    idx=y_descarga.loc[i]
#     display(idx.IdTicker)
    n=0
    #Intentos de descarga:
    for n in range(retries):
        try:     #1er Intento
            #Usando funciones definidas
            last = downloading_yahoo(idx, fecha_inicial, fecha_final)
            #Agregar y esperar para debugging
            a_juntar.append(last)
            time.sleep(espera)
            break #EXITO, pasar a siguiente indice.
        except:   #Segundo intento en caso de falla
            print('Fallo con Ticker')
            time.sleep(espera)
            print('Entrando en modo de fallo')
            if n == retries-1:
                display(idx.IdTicker)
                missing.append(idx.IdTicker)

#Construir data frame final.
df_y = pd.concat(a_juntar)
display(missing)
df_y

  0%|          | 0/259 [00:00<?, ?it/s]

[*********************100%***********************]  1 of 1 completed


Price,Date,Adj Close,Close,High,Low,Open,Volume
Ticker,,AW=F,AW=F,AW=F,AW=F,AW=F,AW=F
0,2025-11-28,110.410004,110.410004,110.68,109.160004,108.080002,374


[*********************100%***********************]  1 of 1 completed


Price,Date,Adj Close,Close,High,Low,Open,Volume
Ticker,,^IXIC,^IXIC,^IXIC,^IXIC,^IXIC,^IXIC
0,2025-11-28,23365.689453,23365.689453,23365.789062,23250.509766,23291.589844,4541070000


...

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['CSC=F']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-11-28 00:00:00 -> 2025-12-31 00:00:00)')


Fallo con Ticker
Entrando en modo de fallo


'CSC=F'

['CC=F',
 'EGPT',
 'ERUS',
 'EWGS',
 'GRES',
 'GXG',
 'JJU',
 'PBS',
 'GLCN',
 'RSX',
 'DC=F',
 'CSC=F']

,DatoFecha,D_Open,D_High,D_Low,D_Close,D_Volume,D_OpenInterest,IdIndiceEspecifico,Descripcion,IdTD,IdTicker,Source
0,2025-11-28,108.080002,110.680000,109.160004,110.410004,374,110.410004,3006,Blmbg Commodity Index,$BCOM(D),AW=F,Yahoo
0,2025-11-28,23291.589844,23365.789062,23250.509766,23365.689453,4541070000,23365.689453,2990,Nasdaq Composite Index,$COMPX,^IXIC,Yahoo
0,2025-11-28,6822.520020,6850.859863,6819.750000,6849.089844,2558540000,6849.089844,2986,S&P 500 Index,$INX,^GSPC,Yahoo
0,2025-11-28,423.000000,448.500000,443.500000,447.750000,78853,447.750000,3011,Corn Continuous Contract [Sep21],@C(D),ZC=F,Yahoo
0,2025-11-28,58.580002,59.639999,58.270000,58.549999,138365,58.549999,3013,Crude Oil Continuous Contract [Sep21],@CL(D),CL=F,Yahoo
0,2025-11-28,64.230003,64.949997,64.419998,62.910000,12848,62.910000,3014,Cotton No. 2 Continuous Contract [Dec21],@CT(D),CT=F,Yahoo
0,2025-11-28,99.540001,99.779999,99.324997,99.407997,13371,99.407997,3001,U.S. Dollar Index Continuous Contract [Sep21],@DX(D),DX=F,Yahoo
0,2025-11-28,4163.399902,4223.899902,4174.600098,4218.299805,149887,4218.299805,3015,Gold Continuous Contract [Aug21],@GC(D),GC=F,Yahoo
0,2025-11-28,5.119500,5.215000,5.055000,5.185500,31863,5.185500,3016,Copper Continuous Contract [Sep21],@HG(D),HG=F,Yahoo
0,2025-11-28,2.268400,2.351000,2.262400,2.333000,18362,2.333000,3017,Heating Oil Continuous Contract [Aug21],@HO(D),HO=F,Yahoo


## Filtro específico para ocasiones especiales

In [61]:
# df_y=df_y.loc[df_y.DatoFecha==df_y.DatoFecha[0].tolist()[0],:].copy()

In [62]:
import pandas as pd
from IPython.display import display

# Ajustar la configuración de visualización de pandas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# ...

solicitados = len(y_descarga)
recibidos = len(df_y)
print('Del total de {} datos solicitados, se recibieron {} datos.'.format(solicitados, recibidos))

df_y_backup = df_y.copy()

print('Indices faltantes:')
display(missing)

print('DataFrame completo:')
display(df_y)


Del total de 259 datos solicitados, se recibieron 247 datos.
Indices faltantes:


['CC=F',
 'EGPT',
 'ERUS',
 'EWGS',
 'GRES',
 'GXG',
 'JJU',
 'PBS',
 'GLCN',
 'RSX',
 'DC=F',
 'CSC=F']

DataFrame completo:


,DatoFecha,D_Open,D_High,D_Low,D_Close,D_Volume,D_OpenInterest,IdIndiceEspecifico,Descripcion,IdTD,IdTicker,Source
0,2025-11-28,108.080002,110.680000,109.160004,110.410004,374,110.410004,3006,Blmbg Commodity Index,$BCOM(D),AW=F,Yahoo
0,2025-11-28,23291.589844,23365.789062,23250.509766,23365.689453,4541070000,23365.689453,2990,Nasdaq Composite Index,$COMPX,^IXIC,Yahoo
0,2025-11-28,6822.520020,6850.859863,6819.750000,6849.089844,2558540000,6849.089844,2986,S&P 500 Index,$INX,^GSPC,Yahoo
0,2025-11-28,423.000000,448.500000,443.500000,447.750000,78853,447.750000,3011,Corn Continuous Contract [Sep21],@C(D),ZC=F,Yahoo
0,2025-11-28,58.580002,59.639999,58.270000,58.549999,138365,58.549999,3013,Crude Oil Continuous Contract [Sep21],@CL(D),CL=F,Yahoo
0,2025-11-28,64.230003,64.949997,64.419998,62.910000,12848,62.910000,3014,Cotton No. 2 Continuous Contract [Dec21],@CT(D),CT=F,Yahoo
0,2025-11-28,99.540001,99.779999,99.324997,99.407997,13371,99.407997,3001,U.S. Dollar Index Continuous Contract [Sep21],@DX(D),DX=F,Yahoo
0,2025-11-28,4163.399902,4223.899902,4174.600098,4218.299805,149887,4218.299805,3015,Gold Continuous Contract [Aug21],@GC(D),GC=F,Yahoo
0,2025-11-28,5.119500,5.215000,5.055000,5.185500,31863,5.185500,3016,Copper Continuous Contract [Sep21],@HG(D),HG=F,Yahoo
0,2025-11-28,2.268400,2.351000,2.262400,2.333000,18362,2.333000,3017,Heating Oil Continuous Contract [Aug21],@HO(D),HO=F,Yahoo


In [63]:
#Preparacion para correr esta seccion de nuevo.
lista_manual = missing

#### Se quiere recibir 257

## En caso de error... buscador de tickets para manual o intentar de nuevo.

Por si no retorna uno de los datos por un fallo inminente utilizar esto para buscar el índice que falló y luego ingresarlo manual en la base de datos

#### Datos no publicados:
- AW=F Para 2022-12-23 !


In [64]:
# missing=['EURUSD=X','GBPUSD=X','MXN=X']
df[df.IdTicker.isin(missing)]

,IdTD,IdTicker,Source,Descripcion,IdIndiceEspecifico
9,@CC(D),CC=F,Yahoo,Cocoa Continuous Contract [Sep21],3012
63,EGPT(HB),EGPT,Yahoo,VanEck Vctrs Egypt Idx,251
75,ERUS(HB),ERUS,Yahoo,iShs MSCI Russia ETF,237
82,EWGS(HB),EWGS,Yahoo,iShs MSCI Germany Sm Cap,234
117,GRES(HB),GRES,Yahoo,IQ Global Resources ETF,2912
119,GXG(HB),GXG,Yahoo,FTSE Colombia 20 ETF,238
167,JJU,JJU,Yahoo,iPath Series B Bloomberg Aluminum Subindex Tot...,3780
188,PBS(HB),PBS,Yahoo,Invesco Dynamic Media ETF,2932
192,PEK,GLCN,Yahoo,VanEck Vectors China Growth Leaders ETF,3774
207,RSX,RSX,Yahoo,VanEck Vctrs Russia ETF,285


In [65]:
#Ejemplo para buscar faltantes:
#query= '''
#SELECT TOP(50) * FROM dbo.TablaDatos Where IdindiceEspecifico = 251 Order By DatoFecha Desc
#'''
#dfOJ = read_data_to_df(query)
#dfOJ[:]

## Unificación final y empacado de datos para base de datos

Aquí se unifican (Cuando está disponible) los índices de Yahoo e Investing y se empacan en el formato de TablaDatos

In [66]:
df_final=df_y.copy()
df_final.to_excel('FinancialMarkets.xlsx',index=False)
# display(df_final)

#Ordenar
df_final=df_final[['IdIndiceEspecifico',
     'DatoFecha',
     'D_Open',
     'D_High',
     'D_Low',
     'D_Close',
     'D_Volume',
     'D_OpenInterest']]
#Renombrar
df_final.columns=['IdIndiceEspecifico',
     'DatoFecha',
     'D_Open',
     'D_High',
     'D_Low',
     'D_Close',
     'D_Volume',
     'D_OpenInterest']
df_final.reset_index(inplace=True,drop=True)
df_f=df_final.copy() #Copia a final
df_f.reset_index(inplace=True,drop=True)


df_f.DatoFecha=df_final.loc[1,'DatoFecha']


In [67]:
df_f

,IdIndiceEspecifico,DatoFecha,D_Open,D_High,D_Low,D_Close,D_Volume,D_OpenInterest
0,3006,2025-11-28,108.080002,110.680000,109.160004,110.410004,374,110.410004
1,2990,2025-11-28,23291.589844,23365.789062,23250.509766,23365.689453,4541070000,23365.689453
2,2986,2025-11-28,6822.520020,6850.859863,6819.750000,6849.089844,2558540000,6849.089844
3,3011,2025-11-28,423.000000,448.500000,443.500000,447.750000,78853,447.750000
4,3013,2025-11-28,58.580002,59.639999,58.270000,58.549999,138365,58.549999
5,3014,2025-11-28,64.230003,64.949997,64.419998,62.910000,12848,62.910000
6,3001,2025-11-28,99.540001,99.779999,99.324997,99.407997,13371,99.407997
7,3015,2025-11-28,4163.399902,4223.899902,4174.600098,4218.299805,149887,4218.299805
8,3016,2025-11-28,5.119500,5.215000,5.055000,5.185500,31863,5.185500
9,3017,2025-11-28,2.268400,2.351000,2.262400,2.333000,18362,2.333000


## Retry para agregar faltantes

In [68]:
for item in ['ERUS', 'RSX']:
    try:
        lista_manual.remove(item)
    except ValueError:
        pass

lista_manual


['CC=F', 'EGPT', 'EWGS', 'GRES', 'GXG', 'JJU', 'PBS', 'GLCN', 'DC=F', 'CSC=F']

In [69]:
# for i in range(0,1):
    #Procesamiento original
idx=lista_manual[-1]
#     display(idx.IdTicker)
n=0
#Intentos de descarga:
for n in range(retries):
    try:     #1er Intento
        #Usando funciones definidas
        last = downloading_yahoo(idx, fecha_inicial, fecha_final)
        #Agregar y esperar para debugging
        a_juntar.append(last)
        time.sleep(espera)
        break #EXITO, pasar a siguiente indice.
    except:   #Segundo intento en caso de falla
        print('Fallo con Ticker')
        time.sleep(espera)
        print('Entrando en modo de fallo')
#         if n == retries-1:
#             display(idx.IdTicker)
#             missing.append(idx.IdTicker)

Fallo con Ticker
Entrando en modo de fallo
Fallo con Ticker
Entrando en modo de fallo
Fallo con Ticker
Entrando en modo de fallo


## Escritura a la base de datos

Se escribe a TablaDatos y TablaDatos_BU hasta que se complete la migración

In [70]:
df_f

,IdIndiceEspecifico,DatoFecha,D_Open,D_High,D_Low,D_Close,D_Volume,D_OpenInterest
0,3006,2025-11-28,108.080002,110.680000,109.160004,110.410004,374,110.410004
1,2990,2025-11-28,23291.589844,23365.789062,23250.509766,23365.689453,4541070000,23365.689453
2,2986,2025-11-28,6822.520020,6850.859863,6819.750000,6849.089844,2558540000,6849.089844
3,3011,2025-11-28,423.000000,448.500000,443.500000,447.750000,78853,447.750000
4,3013,2025-11-28,58.580002,59.639999,58.270000,58.549999,138365,58.549999
5,3014,2025-11-28,64.230003,64.949997,64.419998,62.910000,12848,62.910000
6,3001,2025-11-28,99.540001,99.779999,99.324997,99.407997,13371,99.407997
7,3015,2025-11-28,4163.399902,4223.899902,4174.600098,4218.299805,149887,4218.299805
8,3016,2025-11-28,5.119500,5.215000,5.055000,5.185500,31863,5.185500
9,3017,2025-11-28,2.268400,2.351000,2.262400,2.333000,18362,2.333000


In [71]:
#write2db(df_f,'TablaDatos')
df_f.to_excel('FinancialMarketsBU.xlsx',index=False)